In [1]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import min_weight_full_bipartite_matching
from scipy.sparse.csgraph import maximum_bipartite_matching
import pandas as pd
import timeit
import sys
sys.path.append('/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/MARIO_devo_version/code_0316/') # this is devo version of MARIO
from utils import *
from clustering import *
from matching import *
import matplotlib
import matplotlib.pyplot as plt

In [2]:
df1_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/bonemarrow/bmcite_29000.csv")
df2_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/bonemarrow/levine32_102977.csv")

In [3]:
df1_full

,Unnamed: 0,CD11a,CD11c,CD123,CD127-IL7Ra,CD14,CD16,CD161,CD19,CD197-CCR7,...,CD45RA,CD45RO,CD56,CD57,CD69,CD79b,CD8,HLA.DR,cluster.orig,cluster.term
0,a_AAACCTGAGCTTATCG-1,1.438352,0.912286,0.342896,0.460154,0.363419,0.900484,0.363419,0.256314,0.422575,...,1.213869,0.210040,0.278673,0.760534,0.732842,0.136347,0.826584,2.028515,Prog_RBC,HSPCs
1,a_AAACCTGAGTACATGA-1,2.140502,0.485698,0.060654,1.305309,0.223242,0.446465,0.223242,0.171928,0.223242,...,2.290450,0.060654,0.117838,0.754033,0.405630,0.000000,0.865283,0.559828,CD4 Naive,CD4 T
2,a_AAACCTGCAAACCTAC-1,2.820346,0.706906,0.115421,1.338668,0.218888,0.477309,0.381797,0.218888,0.257453,...,0.618652,1.100769,0.024175,0.430693,0.446474,0.047779,0.550475,0.903470,CD4 Memory,CD4 T
3,a_AAACCTGCAAGGTGTG-1,3.818838,2.830333,0.500528,0.281259,2.186431,0.396891,0.239539,0.281259,0.225237,...,0.671078,0.623863,0.052718,0.553778,1.407460,0.118947,0.671078,1.863981,CD14 Mono,monocyte
4,a_AAACCTGCACGGTAGA-1,1.843356,1.190756,0.535925,0.335635,0.356535,0.569913,0.224034,2.372073,0.610836,...,2.897020,0.137772,0.124818,0.335635,0.870671,0.634609,0.665452,3.048056,Naive B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29001,b_TTTGTCATCCGAGCCA-1,2.544423,0.770929,0.093905,2.006734,0.400303,0.547294,1.019640,0.288743,0.212117,...,0.695267,0.644896,0.129130,0.332049,0.439126,0.093905,0.288743,0.849721,CD4 Memory,CD4 T
29002,b_TTTGTCATCCGTAGGC-1,1.952247,0.632351,0.530885,1.097792,0.271023,0.499897,0.209838,0.271023,0.484036,...,2.585551,0.144665,0.074946,0.575636,0.632351,0.050583,0.467919,0.417948,CD4 Naive,CD4 T
29003,b_TTTGTCATCCTCGCAT-1,3.855561,2.916373,1.497195,0.278553,0.428765,3.909373,0.262212,0.176245,0.211523,...,1.966544,0.245599,0.120877,0.400474,0.294631,0.211523,0.509132,2.063570,CD16 Mono,monocyte
29004,b_TTTGTCATCGCCGTGA-1,1.866485,0.638376,0.233304,1.239093,0.205167,0.792415,0.260672,0.387287,0.531045,...,1.983942,0.387287,0.067725,0.541293,0.422343,0.176215,0.600666,0.561478,CD4 Naive,CD4 T


In [4]:
df2_full

,Unnamed: 0,CD45RA,CD133,CD19,CD22,CD11b,CD4,CD8,CD34,Flt3,...,CD13,CD3,CD61,CD117,CD49d,HLA-DR,CD64,CD41,cluster.orig,cluster.term
0,947,1.200927,-0.018051,-0.013427,0.189780,0.779904,-0.033428,0.044422,-0.010836,0.146383,...,0.176406,0.039531,-0.043190,-0.001330,0.676365,-0.011893,-0.032034,-0.002407,CD16-_NK_cells,NK
1,948,1.455629,-0.009624,0.557719,0.102565,0.588849,-0.023771,0.475519,0.250072,0.175481,...,0.012718,0.276446,0.875353,-0.015510,0.818381,-0.013673,-0.038460,0.003000,CD16-_NK_cells,NK
2,949,1.206293,0.203436,-0.035567,-0.030539,1.113900,0.120961,0.297360,0.278070,0.135693,...,0.011736,0.116658,-0.000758,-0.027312,0.714093,0.011101,-0.027624,-0.034460,CD16-_NK_cells,NK
3,950,0.578248,0.366319,-0.026540,-0.044612,1.352574,-0.042732,0.998905,0.388011,0.282854,...,-0.007292,0.462499,0.505079,-0.014373,0.730525,-0.009912,0.003522,-0.038906,CD16-_NK_cells,NK
4,951,0.263320,0.178058,-0.041834,-0.016937,-0.031140,0.656606,-0.001937,-0.015538,0.190018,...,-0.001830,1.881973,0.217337,-0.027789,0.077744,0.376406,0.049449,-0.011450,CD16-_NK_cells,NK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102972,104180,-0.000106,-0.030641,1.432347,-0.044946,-0.016534,0.389503,-0.031912,2.178695,-0.037439,...,1.551640,-0.020972,0.188846,-0.002144,1.115652,2.373524,-0.004620,-0.051592,Pro_B_cells,B
102973,104181,0.692668,-0.037335,1.639063,0.286325,-0.036985,-0.045311,0.071127,1.651417,0.126024,...,0.426976,-0.017870,-0.029213,-0.031301,1.653418,4.367032,0.062683,0.158656,Pro_B_cells,B
102974,104182,-0.036795,-0.014477,1.637975,-0.021794,-0.020169,-0.035501,0.064389,2.626539,-0.027089,...,-0.000175,-0.030173,-0.015220,-0.034755,1.083173,3.541526,0.110382,0.108349,Pro_B_cells,B
102975,104183,0.308817,0.075762,1.455130,0.042576,-0.049737,-0.033189,-0.047487,2.413536,0.526259,...,1.142426,-0.015380,-0.016644,-0.047522,0.432565,3.882030,0.058852,0.185295,Pro_B_cells,B


In [7]:
df1_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/pbmc/10xciteseq_5107_clean.csv")
df2_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/pbmc/felixcytof_38866.csv")

In [8]:
df1_full

,Unnamed: 0,X,CD3,CD4,CD8a,CD11b,CD14,CD15,CD16,CD19,...,CD127,CD137,CD197,CD274,CD278,CD335,PD.1,HLA.DR,TIGIT,cluster.info
0,1,AAACCCAAGAGACAAG-1,25,164,16,3011,696,19,10,6,...,2,8,6,1,10,4,7,573,10,CD14 Mono
1,2,AAACCCAAGGCCTAGA-1,959,720,8,12,12,9,4,5,...,37,0,10,3,121,5,17,15,3,naive CD4 T
2,3,AAACCCAGTCGTGCCA-1,942,1647,21,11,13,8,7,0,...,117,7,15,4,92,12,39,11,3,mem CD4 T
3,4,AAACCCATCGTGCATA-1,802,1666,5,11,9,8,3,3,...,157,3,3,5,44,6,15,19,3,mem CD4 T
4,5,AAACGAAAGAGTGACC-1,18,151,15,3018,583,17,1,5,...,4,2,10,3,3,5,4,441,3,CD14 Mono
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,5103,TTTGTTGCAGCACAAG-1,693,9,737,20,15,6,1,1,...,7,1,12,3,146,5,34,23,14,CD8 T
5103,5104,TTTGTTGCAGTCTTCC-1,402,1417,8,15,9,14,2,2,...,145,1,2,1,65,8,11,6,2,mem CD4 T
5104,5105,TTTGTTGCATGGCCCA-1,401,1,222,7,17,6,1,2,...,14,0,1,2,40,0,8,40,15,CD8 T
5105,5106,TTTGTTGCATTGCCGG-1,6,46,3,1027,382,1,4,4,...,0,3,7,0,3,1,3,184,1,CD14 Mono


In [9]:
df1_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/Cytof-Xspecies-IFNG/wcct4d_term_srcluster120k.csv")
df2_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/Cytof-Xspecies-IL4/zac_human_il4_120k.csv")

In [10]:
df1_full

,Unnamed: 0,index,CD38,CD235a,CD45,STAT3,CD61,CD7,CD33,CD11c,...,STAT4,CCR7,4EBP1,CD19,IgM,CD56,HLA-DR,CD27,cluster.orig,cluster.sr
0,108264,207814,1.624857,1.265603,4.769958,4.008287,3.255571,1.508704,2.671402,2.453076,...,1.675040,2.027956,0.827843,0.622960,2.027437,2.003590,0.079178,0.468482,CD66+,Neutrophil
1,71342,170892,1.970554,0.789837,5.124874,4.291483,3.439626,1.922451,2.129142,4.102474,...,1.779030,3.510430,2.391465,1.781067,3.341147,3.540580,7.525417,1.177626,cMCs,cMC
2,25368,124918,0.900609,0.122020,3.032797,3.077967,2.681875,0.566141,0.888844,1.201301,...,2.065188,2.006705,1.056867,0.328347,1.267000,2.154552,0.054257,-0.005881,CD66+,Neutrophil
3,21993,121543,0.373601,0.461866,4.473738,2.756498,2.704668,0.595601,0.775014,2.457485,...,0.958310,1.614957,0.976140,0.380008,1.561223,1.257547,0.348256,-0.005881,CD66+,Neutrophil
4,32201,131751,1.066518,0.372582,3.395520,2.838310,2.940574,0.935336,2.114436,1.742743,...,1.910953,2.942250,2.059519,0.072798,1.919513,1.902844,0.238978,0.028130,CD66+,Neutrophil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102142,40717,140267,0.603218,0.976469,4.927455,3.180913,1.590192,1.267520,1.272323,2.356473,...,1.123470,1.898927,1.085053,0.334495,1.587882,1.964716,0.983924,0.634109,CD66+,Neutrophil
102143,26468,126018,0.866552,0.506608,3.408506,3.143847,1.712921,0.861131,1.214093,2.872294,...,0.896175,3.047261,0.440790,0.072798,2.727452,0.987711,0.139172,0.351407,CD66+,Neutrophil
102144,42585,142135,0.463014,1.551874,4.615100,3.439974,2.682508,1.217535,1.290533,2.441164,...,0.886507,2.099920,1.175033,0.228877,2.077633,1.305567,0.773142,0.654142,CD66+,Neutrophil
102145,108725,208275,1.006638,0.424263,4.482774,3.839822,3.862832,0.632695,1.652680,0.863266,...,0.908198,1.952903,0.989378,0.367127,1.910332,1.275954,0.749725,-0.005881,CD66+,Neutrophil


In [11]:
df2_full

,Unnamed: 0,X,index,CD235a,CD45,STAT3,CD61,CD7,CD33,CD11c,...,Ki67,STAT4,CCR7,X4EBP1,CD19,IgM,CD56,HLA.DR,cluster.orig,cluster.sr
0,1,38613,742513,0.902710,4.404464,2.361583,3.745740,1.802735,2.630472,3.296863,...,2.347268,1.720599,4.291052,1.758669,1.003926,4.795647,3.809151,1.046528,Neutrophils,Neutrophil
1,2,203412,907312,0.526088,3.936984,3.045701,3.619406,1.006511,3.597066,2.721330,...,3.452709,2.587980,4.366969,3.579882,1.247663,3.547551,4.348586,3.798204,CD11b+CD16-,MC
2,3,102928,806828,2.405628,3.964641,3.315373,4.935642,4.807839,1.764254,0.966053,...,1.562056,0.670165,3.152019,1.586936,0.930764,3.996795,3.785055,0.769030,CD8+Tcells,CD8 T
3,4,45807,749707,1.615307,4.072375,2.703688,3.551730,0.593239,2.553803,2.124498,...,0.097712,0.836084,2.883206,0.000000,0.337864,5.121729,2.473483,0.047805,Neutrophils,Neutrophil
4,5,49162,753062,0.567513,4.033425,2.805915,3.283802,1.568305,2.544401,2.562311,...,1.445164,2.120695,4.149288,2.014223,0.911614,4.403591,3.603514,0.079058,Neutrophils,Neutrophil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108533,108534,14323,718223,0.651545,5.469732,2.879109,3.649457,1.148312,1.182382,0.873185,...,2.757102,2.409337,4.372058,2.551897,1.659908,4.470252,4.541191,1.276513,CD11b+CD16-,CD4 T
108534,108535,120527,824427,2.092353,3.982072,3.283582,3.548776,1.689356,3.302763,2.864958,...,0.344407,0.011870,2.652475,0.298635,0.000000,4.312362,2.734383,3.158011,CD11b+CD16-,MC
108535,108536,169747,873647,0.779588,3.869256,2.509809,3.477280,1.398048,2.454119,2.693995,...,1.206084,1.021728,3.816998,1.175571,1.079894,4.622626,3.144336,0.290712,Neutrophils,Neutrophil
108536,108537,152884,856784,0.606124,3.466284,3.701073,4.720428,1.951402,2.978450,3.270813,...,1.925338,2.276037,3.724477,0.969638,1.297973,3.758332,3.032005,0.371206,Neutrophils,Neutrophil


In [6]:
df1_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/murine_spleen/MurineCodex_mario_matched.csv")
df2_full = pd.read_csv("/home/bkzhu/SNE-multi/figure_rcode/figure_related_code_submit/data/murine_spleen/MurineCiteseq_mario_matched.csv")

In [3]:
df1_full

,Unnamed: 0,Ly6C,TCR,Ly6G,CD19,CD169,CD106,CD3,CD1632,CD8a,...,CD35,CD2135,nucl,NKp46,cluster.sr,cluster.term,cellLabelInImage,PointNum,cite_cluster1,cite_cluster2
0,1,319.198084,896.135570,321.788827,1995.661413,1255.585596,1795.921321,1133.312083,3967.976873,1845.643158,...,1321.024583,573.092933,4613.719016,440.965341,B,B,961,6,Mature B,B
1,2,115.987414,287.768062,288.656546,301.406059,414.497258,0.000000,234.475801,65.719595,258.634470,...,844.171362,496.112861,3497.056247,80.692433,B,B,362,20,Mature B,B
2,3,1094.918305,2326.377573,693.662375,0.000000,1158.822912,411.274517,2099.964485,0.000000,0.000000,...,0.000000,0.000000,7643.301108,2366.762776,NK,NK,1356,53,Ly6-high mono,Mono
3,4,131.188445,0.000000,471.060342,2870.333672,2591.987031,906.413055,572.042713,1523.332404,2579.832043,...,5303.800846,3723.160251,1014.835868,360.969993,gc B,B,184,35,MZ B,B
4,5,327.199485,1384.577739,378.985215,3510.188775,420.960038,1754.165666,1423.851464,3008.257372,1907.276443,...,1233.302885,604.222616,6889.571176,244.025843,B,B,1304,52,Mature B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48327,48328,120.964857,0.000000,115.639316,1126.481825,533.290664,0.000000,43.764753,345.554217,776.482269,...,262.997885,316.630313,2420.658924,0.000000,B,B,593,33,Mature B,B
48328,48329,251.009908,7355.283227,227.477535,1012.687276,1218.440209,1298.036902,2206.205200,666.423140,14069.415942,...,844.211931,291.031115,4972.768538,626.435675,CD8 T,CD8 T,244,5,CD8 T,CD8 T
48329,48330,136.461491,913.454273,249.978426,1753.048511,990.139045,669.705328,902.333783,559.548051,2824.116951,...,1021.728173,690.977339,6362.101662,386.303757,B,B,1429,12,Mature B,B
48330,48331,0.000000,0.000000,89.098238,407.200728,707.591094,2248.399401,653.691815,3785.525900,2331.429257,...,485.963550,184.288419,4838.635352,4880.538161,NK,NK,754,8,NK,NK


In [7]:
df2_full.iloc[:,4:212]

,CD103,CD105,CD106,CD107a.LAMP.1.,CD115.CSF.1R.,CD117.c.Kit.,CD11a,CD11b,CD11c,CD120b.TNFRTypeII.p75.,...,IsotypeCtrlMouseIgG1,IsotypeCtrlMouseIgG2a,IsotypeCtrlMouseIgG2b,IsotypeCtrlRatIgG1,IsotypeCtrlRatIgG2a,IsotypeCtrlRatIgG2b,IsotypeCtrlRatIgG2c,HTO,HTO.1,cluster.orig
0,4,5,3,2,4,2,43,4,1,5,...,4,1,1,4,2,5,4,1,125,Mature B
1,2,5,8,3,2,1,103,12,1,2,...,3,2,1,5,2,1,0,2,182,NK
2,13,2,6,1,2,2,16,3,1,5,...,2,1,0,15,1,2,0,2,82,Mature B
3,3,1,4,4,5,2,109,3,2,2,...,3,3,2,6,3,2,1,3,275,MZ B
4,8,1,3,2,0,2,41,4,3,3,...,4,2,1,6,3,0,1,4,83,B1 B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36668,5,1,2,3,0,2,57,1,4,2,...,0,0,0,9,5,1,1,2,322,Mature B
36669,6,0,2,2,2,4,52,5,2,2,...,3,3,1,7,4,3,3,1,98,MZ B
36670,13,1,3,1,3,7,71,5,1,3,...,0,3,0,8,3,2,2,0,188,CD8 T
36671,2,2,3,4,0,1,21,2,0,1,...,0,1,1,1,4,1,1,2,97,Transitional B


In [8]:
df2_full

,Unnamed: 0,cellID,X4.1BBLigand.CD137L.,CD102,CD103,CD105,CD106,CD107a.LAMP.1.,CD115.CSF.1R.,CD117.c.Kit.,...,IsotypeCtrlMouseIgG2a,IsotypeCtrlMouseIgG2b,IsotypeCtrlRatIgG1,IsotypeCtrlRatIgG2a,IsotypeCtrlRatIgG2b,IsotypeCtrlRatIgG2c,HTO,HTO.1,cluster.orig,cluster.info
0,1,GATGAGGTCGCCTCTA-1,3,63,4,5,3,2,4,2,...,1,1,4,2,5,4,1,125,Mature B,B
1,2,CGATGCGAGGACAGCT-2,2,26,2,5,8,3,2,1,...,2,1,5,2,1,0,2,182,NK,NK
2,3,CACATGATCGACGCTG-2,5,32,13,2,6,1,2,2,...,1,0,15,1,2,0,2,82,Mature B,B
3,4,TGCCGAGCACAAGGTG-1,1,118,3,1,4,4,5,2,...,3,2,6,3,2,1,3,275,MZ B,B
4,5,ATCGTAGAGCTTTCCC-1,3,45,8,1,3,2,0,2,...,2,1,6,3,0,1,4,83,B1 B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36668,36669,GTGTAACCAGACCGCT-2,3,39,5,1,2,3,0,2,...,0,0,9,5,1,1,2,322,Mature B,B
36669,36670,GCTTGGGGTCTGTAAC-1,5,25,6,0,2,2,2,4,...,3,1,7,4,3,3,1,98,MZ B,B
36670,36671,GACTGATGTCCGGTGT-2,2,11,13,1,3,1,3,7,...,3,0,8,3,2,2,0,188,CD8 T,CD8 T
36671,36672,GTCAAACAGTGGAAAG-1,6,30,2,2,3,4,0,1,...,1,1,1,4,1,1,2,97,Transitional B,B
